In [1]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                       
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 114 kB in 2s (70.8 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1).
libxext6 is already the newest version (2:1.3.4-0ubuntu1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [2]:
!pip install seaborn

In [3]:
!nvidia-smi 

Sun Jul  3 15:11:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:83:00.0 Off |                  Off |
| N/A   36C    P0    50W / 250W |      0MiB / 24451MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [5]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
#https://keras.io/api/applications/

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def generators(shape): 
    imgdatagen = ImageDataGenerator(
        rotation_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.5,
        horizontal_flip = True, 
        fill_mode='nearest',
        validation_split = 0.2,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        './my_dataset_big/',
        target_size = (height, width), 
        batch_size = batch_size,
        class_mode='categorical',
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        './my_dataset_big/',
        target_size = (height, width), 
        batch_size = batch_size,
        class_mode='categorical',
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [8]:
batch_size =5

In [9]:
model_name='EfficientNetB7'
base_model=tf.keras.applications.EfficientNetB7(include_top=False,input_shape=(600,600,3), weights="imagenet", pooling='max') 
x=base_model.output
x=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.45, seed=123)(x)        
output=Dense(3, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(), loss='categorical_crossentropy', metrics=['accuracy']) 


2022-07-03 15:12:01.388890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23680 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:83:00.0, compute capability: 6.1


In [10]:
#model.summary()

In [11]:
train_dataset, val_dataset = generators((600,600))



Found 6055 images belonging to 3 classes.
Found 1512 images belonging to 3 classes.


In [12]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau


earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./EfficientnetB7{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')


In [13]:
print(tf.__version__)

2.8.2


In [14]:
model = tf.keras.models.load_model('./'+'EfficientnetB7_base.hdf5')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 600, 600, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 600, 600, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 600, 600, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

In [ ]:

history = model.fit(
    train_dataset, 
    validation_data = val_dataset,
    epochs=50,
    callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
    )

Epoch 1/50


2022-07-03 15:12:50.502788: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


1211/1211 [==============================] - 2504s 2s/step - loss: 0.4422 - accuracy: 0.9174 - val_loss: 0.2998 - val_accuracy: 0.9709 - lr: 0.0010
Epoch 2/50
1211/1211 [==============================] - 2512s 2s/step - loss: 0.3768 - accuracy: 0.9407 - val_loss: 0.3488 - val_accuracy: 0.9603 - lr: 0.0010
Epoch 3/50
1211/1211 [==============================] - 2496s 2s/step - loss: 0.3214 - accuracy: 0.9539 - val_loss: 0.2711 - val_accuracy: 0.9762 - lr: 0.0010
Epoch 4/50
 104/1211 [=>............................] - ETA: 35:42 - loss: 0.2877 - accuracy: 0.9615